<a href="https://colab.research.google.com/github/ToshikiShimizu/PyTorch/blob/master/intro_dev/chapter4/4-4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torchvision
!pip install torch
import torch
from torch import nn, optim
from torch.utils.data import(Dataset, DataLoader, TensorDataset)
import tqdm

    100% |████████████████████████████████| 61kB 2.7MB/s 
    100% |████████████████████████████████| 2.0MB 6.1MB/s 
    100% |████████████████████████████████| 519.5MB 28kB/s 
tcmalloc: large alloc 1073750016 bytes == 0x5a1a2000 @  0x7fe21ffdd2a4 0x594e17 0x626104 0x51190a 0x4f5277 0x510c78 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f6070 0x510c78 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f6070 0x4f3338 0x510fb0 0x5119bd 0x4f6070
  Found existing installation: Pillow 4.0.0
    Uninstalling Pillow-4.0.0:
      Successfully uninstalled Pillow-4.0.0


In [2]:
!ls

sample_data


In [3]:
!wget http://vis-www.cs.umass.edu/lfw/lfw-deepfunneled.tgz
!tar xf lfw-deepfunneled.tgz


--2018-11-13 12:52:02--  http://vis-www.cs.umass.edu/lfw/lfw-deepfunneled.tgz
Resolving vis-www.cs.umass.edu (vis-www.cs.umass.edu)... 128.119.244.95
Connecting to vis-www.cs.umass.edu (vis-www.cs.umass.edu)|128.119.244.95|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 108761145 (104M) [application/x-gzip]
Saving to: ‘lfw-deepfunneled.tgz’

lfw-deepfunneled.tg 100%[===================>] 103.72M  42.7MB/s    in 2.4s    

2018-11-13 12:52:04 (42.7 MB/s) - ‘lfw-deepfunneled.tgz’ saved [108761145/108761145]



In [0]:
!mkdir train
!mkdir test
!mv lfw-deepfunneled/[A-W]* train
!mv lfw-deepfunneled/[X-Z]* test

In [0]:
from torchvision.datasets import ImageFolder
from torchvision import transforms

In [0]:
class DownSizedPairImageFolder(ImageFolder):
  def __init__(self, root, transform=None, large_size=128, small_size=32, **kwds):
    super().__init__(root, transform=transform, **kwds)
    self.large_resizer = transforms.Resize(large_size)
    self.small_resizer = transforms.Resize(small_size)
  def __getitem__(self, index):
    path, _ = self.imgs[index]
    img = self.loader(path)
    large_img = self.large_resizer(img)
    small_img = self.small_resizer(img)
    if self.transform is not None:
      large_img = self.transform(large_img)
      small_img = self.transform(small_img)
    return small_img, large_img
    

In [0]:
train_data = DownSizedPairImageFolder("train", transform = transforms.ToTensor())
test_data = DownSizedPairImageFolder("test", transform = transforms.ToTensor())
batch_size = 32
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=4)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=4)


In [0]:
net = nn.Sequential(
  nn.Conv2d(3,256,4,2,1),
  nn.ReLU(),
  nn.BatchNorm2d(256),
  nn.Conv2d(256,512,4,2,1),
  nn.ReLU(),
  nn.BatchNorm2d(512),
  nn.ConvTranspose2d(512,256,4,2,1),
  nn.ReLU(),
  nn.BatchNorm2d(256),
  nn.ConvTranspose2d(256,128,4,2,1),
  nn.ReLU(),
  nn.BatchNorm2d(128),
  nn.ConvTranspose2d(128,64,4,2,1),
  nn.ReLU(),
  nn.BatchNorm2d(64),
  nn.ConvTranspose2d(64,3,4,2,1),
)

In [0]:
import math
def psnr(mse, max_v=1.0):
  return 10 * math.log10(max_v**2 / mse)
def eval_net(net, data_loader, device="cpu"):
  net.eval()
  ys = []
  ypreds = []
  for x, y in data_loader:
    x = x.to(device)
    y = y.to(device)
    with torch.no_grad():
      y_pred = net(x)
    ys.append(y)
    y_preds.append(y)
  ys = torch.cat(ys)
  ypreds = torch.cat(ypreds)
  score = nn.functional.mse_loss(ypreds, ys).item()
  return score

In [0]:
def train_net(net, train_loader, test_loader, optimizer_cls=optim.Adam, loss_fn=nn.MSELoss(), n_iter=10, device="cpu"):
  train_losses = []
  train_acc = []
  val_acc = []
  optimizer = optimizer_cls(net.parameters())
  for epoch in range(n_iter):
    running_loss = 0.0
    net.train()
    n = 0
    score = 0
    for i, (xx, yy) in tqdm.tqdm(enumerate(train_loader), total=len(train_loader)):
      xx = xx.to(device)
      yy = yy.to(device)
      y_pred = net(xx)
      loss = loss_fn(y_pred, yy)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      running_loss += loss.item()
      n += len(xx)
    train_losses.append(running_loss / len(train_loader))
    val_acc.append(eval_net(net, test_loader, device))
    print (epoch, train_losses[-1], psnr(train_losses[-1]),psnr(val_acc[-1], flush=True))

In [11]:
net.to("cuda:0")
train_net(net, train_loader, test_loader, device="cuda:0")

  0%|          | 0/409 [00:00<?, ?it/s]

AttributeError: ignored

In [0]:
from torchvision.utils import save_image

random_test_loader = DataLoader(test_data, batch_size=4, shuffle=True)
it = iter(random_test_loader)
x, y = next(it)